In [ ]:
# Loader
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader

# Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Store
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma

# Tokenization
from transformers import AutoTokenizer

# Language model
from transformers import RobertaForSequenceClassification

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset, TensorDataset
from torch.nn.functional import softmax

# Additional libraries
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import time
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

#### **Load and split new document**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 100, add_start_index = False) # splits the text into chunks
loader = PyPDFLoader("/home/ssever/ContraDoc/data/Inject/Press_release_q1_2024.pdf").load()
split_text = text_splitter.split_documents(loader)

In [ ]:
chunks = [doc.page_content for doc in split_text]
print(f"Number of chunks in new document: {len(chunks)}")

#### **Similarity search**

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")
db = Chroma(persist_directory="/home/ssever/ContraDoc/src/vectorization/vector_store/no_kb_langchain", embedding_function=embedding_function)

In [ ]:
knowledge_chunks = []
k = 1
for chunk in chunks:
    docs = db.similarity_search_with_score(chunk, k=k)
    #print(docs[0][1])
    #if docs[0][1] <= 0.5:
    #    print(docs[0][0])

    for doc, score in docs:
        knowledge_chunks.append(doc.page_content)

(f"Number of similar chunks in knowledge base: {len(knowledge_chunks)}")

#### **Create and clean pairs**

In [ ]:
# Cleaning each premise
cleaned_premises = [
    re.sub(r'\s+\.', '.',  # Remove space before period
           re.sub(r'\s+,', ',',  # Remove space before comma
                  re.sub(r'\n', '', chunk)))  # Remove new lines
    for chunk in chunks]

print(len(cleaned_premises))

In [ ]:
# Cleaning each hypothesis
cleaned_hypotheses = [
    re.sub(r'\s+\.', '.',  # Remove space before period
           re.sub(r'\s+,', ',',  # Remove space before comma
                  re.sub(r'\n', '', chunk)))  # Remove new lines
    for chunk in knowledge_chunks]

print(len(cleaned_hypotheses))

In [ ]:
#similar_pairs = list(zip(cleaned_premises, cleaned_hypotheses))
#print(f"Number of chunk pairs: {len(similar_pairs)}")

In [ ]:
# Result list
similar_pairs = []
num_pairs = k
# Loop through the first list
for i in range(len(cleaned_premises)):
    # Create pairs for the specified number of corresponding strings from list2
    for j in range(num_pairs):
        pair = (cleaned_premises[i], cleaned_hypotheses[num_pairs*i + j])
        similar_pairs.append(pair)

#### **Classification of pairs**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device type:', device)
print('There are %d GPU(s) available.' % torch.cuda.device_count())
print('GPU is:', torch.cuda.get_device_name(0))

In [ ]:
model_path = '/home/ssever/ContraDoc/model/roberta-base.pt'
#model_path = '/home/ssever/ContraDoc/model/xlm-roberta-base-snli-mnli-anli-xnli.pt'
#model_path = '/home/ssever/ContraDoc/model/roberta-large.pt'

tokenizer_path = 'roberta-base'
#tokenizer_path = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
#tokenizer_path = 'roberta-large'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = torch.load(model_path)
model.to(device)

MAX_LENGTH = 256

# Function for encoding input data
def encode_sets(data, tokenizer):
    kwargs = { 'truncation': True,
    'max_length': MAX_LENGTH,
    'padding': 'max_length',
    'return_attention_mask': True, 
    'return_token_type_ids': True     
    }
    #datalist = list(zip(data['premise'], data['hypothesis']))
    tokenized = tokenizer.batch_encode_plus(data,**kwargs)
    input_ids = torch.LongTensor(tokenized.input_ids)
    attention_masks = torch.LongTensor(tokenized.attention_mask)
    token_type_ids = torch.LongTensor(tokenized.token_type_ids)
    return input_ids, attention_masks, token_type_ids


# Encode similar pairs
input_ids, attention_masks, token_type_ids = encode_sets(similar_pairs, tokenizer)
pairs_tensor = TensorDataset(input_ids, attention_masks, token_type_ids)
pairs_dataloader = DataLoader(pairs_tensor, sampler=SequentialSampler(pairs_tensor), batch_size=len(similar_pairs))

model.eval()

for batch in pairs_dataloader:

    with torch.no_grad():
        input_ids, attention_masks, token_type_ids = (batch[0].to(device), 
                                                      batch[1].to(device),
                                                      batch[2].to(device))
        start_time = time.time()
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_masks)
        end_time = time.time()

    logits = outputs.logits
    model_preds = logits.detach().cpu().numpy()

    # Calculate inference time
    inference_time = end_time - start_time
    print(f"\nInference time: {inference_time:.4f} seconds\n")
    predictions = np.argmax(model_preds, axis=1)

# target names according to 3 label or 11 label model
if model_path == '/home/ssever/ContraDoc/model/xlm-roberta-base-snli-mnli-anli-xnli.pt':
    target_names = [
    "entailment",
    "neutral",
    "contradiction"
]
else:
    target_names = [
    "antonym",
    "entailment",
    "factive_antonym",
    "factive_embedding_verb",
    "lexical",
    "negation",
    "neutral",
    "numeric",
    "structure",
    "temporal",
    "worldknowledge"
]
    
# Use Counter to count occurrences
count = {target_names[key]: value for key, value in Counter(predictions).items()}

# Printing the count of each number
print(f"Predictions:\n{count}")

# count all predicted labels
total_count = sum(count.values())

# check whether label entailment has been predicted at least once and count the amount
if 'entailment' in count.keys():
    entailment_share = (count["entailment"] / total_count) * 100
else:
    entailment_share = 0

# check whether label neutral has been predicted at least once and count the amount
if 'neutral' in count.keys():
    neutral_share = (count["neutral"] / total_count) * 100
else:
    neutral_share = 0

# check whether a contradiction label has been predicted at least once count the amount
keys = {'entailment', 'neutral'}
contradiction_keys = set(count.keys()) - keys
if contradiction_keys:
    contradiction_labels =  [label for label in count if label not in ["entailment", "neutral"]]
    contradiction_count = sum(count[label] for label in contradiction_labels)
    contradiction_share = (contradiction_count / total_count) * 100
else:
    contradiction_share = 0

print(f"\nDistribution:\nEntailment: {entailment_share:.2f}%, Neutral: {neutral_share:.2f}%, Contradictions: {contradiction_share:.2f}%")

In [ ]:
# calculate probabilty for each predicted class
probabilities = softmax(logits, dim=1)
#probabilities = [[round(max(value), 5), target_names[value.index(max(value))]] for value in probabilities.tolist()]
probabilities = [round(max(value), 2) for value in probabilities.tolist()]

# join pairs with label and probability
labels = [target_names[number] for number in predictions]
classifications = list(zip(similar_pairs, labels))
list(zip(classifications, probabilities))
entailment_neutral = [[[probability, label], premise, hypothesis] for (((premise, hypothesis), label), probability) in list(zip(classifications, probabilities)) if label == 'entailment' or label == 'neutral']
#contradictions = [[[probability, label], premise, hypothesis] for (((premise, hypothesis), label), probability) in list(zip(classifications, probabilities)) if label != 'entailment' and label != 'neutral']
#classifications = [[[probability, label], premise, hypothesis] for (((premise, hypothesis), label), probability) in list(zip(classifications, probabilities))]
#for number, pair in enumerate(contradictions):
#    print(number, pair)
print(len(entailment_neutral))
#print(len(contradictions))

#### **Store results in persistent variable**

In [ ]:
langchain_chunks = [[premise, hypothesis, label] for [[probability, label], premise, hypothesis] in entailment_neutral]
#langchain_chunks = [[premise, hypothesis, label] for [[probability, label], premise, hypothesis] in contradictions]
%store langchain_chunks

#### **Evaluation graph**

In [ ]:
# Data
categories = ['A', 'B', 'C', 'D', 'E']
values = [6, 2, 0, 6, 5]

# Creating the bar diagram
plt.figure(figsize=(6, 3))
plt.bar(categories, values, color='blue')
plt.title('Bar Diagram')
plt.xlabel('Labels')
plt.ylabel('Values')

# Show the plot
plt.show()